# Text Mining Project
test code

**Load drive path**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Text Mining/project"  # your directory in google drive
os.chdir(path)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim
import spacy

nlp = spacy.load('en_core_web_sm')
# frame = pd.read_json('News_Category_Dataset_v2.json',encoding='utf-8',orient='records',lines=True)

**Import**

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers,layers,losses
from collections import Counter
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pydot
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Conv1D, Flatten, Dense, Lambda, LSTM, concatenate,SimpleRNN,SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from numpy.ma.extras import average
from tensorflow.python.keras.backend import mean
import pickle
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

**new IMDb data**

In [ ]:
IMDB_training_data = pd.read_csv('IMDB Dataset.csv')
IMDB_training_data = pd.DataFrame(IMDB_training_data)

IMDB_training_data

**Preprocessing**

In [ ]:
# remove punctuation
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.lower()

## remove stop words and number    
def preprocess(text):
    # TODO: Replace the next line with your own code.
    doc = nlp(text)
    y = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    # y = [token.lemma_ for token in doc if not token.is_stop and token.lemma_.isalpha()]
    return y


In [ ]:
# preprocessing and save the data
tt = IMDB_training_data['review'].apply(clean_text).apply(preprocess)
pickle.dump((tt),open('tokenized_IMDB.pkl','wb'))

In [ ]:
# load the preprocessed data
IMDB_token = pickle.load(open('tokenized_IMDB.pkl', 'rb'))
for i in range(len(IMDB_token)):
  IMDB_token[i] = ' '.join(IMDB_token[i])
# print((IMDB_token))
tt = IMDB_training_data
tt['review'] = IMDB_token
print(tt['review'][0])
training_data, test_data, y_training, y_test = train_test_split(IMDB_token,\
                                                      IMDB_training_data['sentiment'],\
                                                      test_size=0.1,\
                                                      # stratify = train_Y.tolist(),\
                                                      shuffle=True,\
                                                      random_state=1)

large_training_data, training_data, large_y_training, y_training = train_test_split(training_data,\
                                                      y_training,\
                                                      test_size=0.5,\
                                                      # stratify = train_Y.tolist(),\
                                                      shuffle=True,\
                                                      random_state=1)

training_data, val_data, y_training, y_val = train_test_split(training_data,\
                                                      y_training,\
                                                      test_size=0.1,\
                                                      # stratify = train_Y.tolist(),\
                                                      shuffle=True,\
                                                      random_state=1)

# len(training_data)

In [ ]:
# function to plot model training process
def plot_modelfit_process(model_history, title):
  n = len(model_history.history['accuracy'])
  fig, ax = plt.subplots(2, 1, figsize=(8, 8))
  ax[0].set_title('Accuracy')
  ax[0].plot(model_history.history['accuracy'])
  ax[0].plot(model_history.history['val_accuracy'])
  ax[0].axvline(x=np.where(model_history.history['val_accuracy'] == np.max(model_history.history['val_accuracy']))[0][0],
                color='k', linestyle='--')
  ax[0].set_xlabel("Epochs")
  ax[0].set_xticks(range(n))
  ax[0].legend(['train', 'validation'])

  ax[1].set_title('Loss')
  ax[1].plot(model_history.history['loss'])
  ax[1].plot(model_history.history['val_loss'])
  ax[1].axvline(x=np.where(model_history.history['val_loss'] == np.min(model_history.history['val_loss']))[0][0],
                color='k', linestyle='--')
  ax[1].set_xlabel("Epochs")
  ax[1].set_xticks(range(n))
  ax[1].legend(['train', 'validation'])
  fig.tight_layout(pad=2.0)
  plt.show()
  fig.savefig(path+title)

**Data description**

In [ ]:
print('-------Data--------')
print(tt['sentiment'].value_counts())
print(len(tt['sentiment']))
print('-------------------')

max_sentence_length  = max([len(str(text).split()) for text in tt['review']])

print('Max Sentence Length :'+str(max_sentence_length))

from IPython.core.pylabtools import figsize
label,counts = np.unique(tt['sentiment'],return_counts=True)
print(label)
print(counts)

ind_pos = np.where(tt['sentiment'] == 'positive')
ind_neg = np.where(tt['sentiment'] == 'negative')
# print(training_data.iloc[list(ind_pos)[0]])
# print(len(list(ind_pos)))
fig = plt.figure(1)
plt.bar(label,counts)
plt.xlabel("class label", labelpad=18)
plt.ylabel("number of documents", labelpad=18)
plt.title("Data Label Distribution", y=1.02);
fig.savefig(path+'/figures/total_label_distr.png')

text_length  = [len(str(text).split()) for text in tt['review']]

fig = plt.figure(2)
plt.hist(x=text_length,bins=80,
        color="steelblue",
        edgecolor="black")
plt.xlabel("text length", labelpad=18)
plt.ylabel("frequency", labelpad=18)
plt.title("Text Length Distribution", y=1.02);
fig.savefig(path+'/figures/total_text_len.png')

positive_text_length  = [len(str(text).split()) for text in tt['review'].iloc[list(ind_pos)[0]]]  
negative_text_length  = [len(str(text).split()) for text in tt['review'].iloc[list(ind_neg)[0]]]   

fig,ax = plt.subplots(1,2, figsize=(12,6))
fig.suptitle('Text Length')
ax[0].hist(x=positive_text_length,bins=80,color="steelblue",edgecolor="black")
ax[0].set(xlabel = 'Length', ylabel = 'frequency', title = 'Positive Reviews')
ax[1].hist(x=negative_text_length,bins=80,color="steelblue",edgecolor="black")
ax[1].set(xlabel = 'Length', ylabel = 'frequency', title = 'Negative Reviews')
fig.savefig(path+'/figures/two_class_text_len.png')



In [ ]:
print('-------Train data--------')
print(y_training.value_counts())
print(len(y_training))
print('-------------------------')

# training_data = training_data.apply(clean_text)
max_train_sentence_length  = max([len(str(text).split()) for text in training_data])
# print(np.max(max_train_sentence_length))

print('-------Test data--------')
print(y_test.value_counts())
print(len(y_test))
print('-------------------------')

# test_data = test_data.apply(clean_text)
max_test_sentence_length  = max([len(str(text).split()) for text in test_data])

print('Train Max Sentence Length :'+str(max_train_sentence_length))
print('Test Max Sentence Length :'+str(max_test_sentence_length))

In [ ]:
from IPython.core.pylabtools import figsize
label,counts = np.unique(y_training,return_counts=True)
print(label)
print(counts)

ind_pos = np.where(y_training == 'positive')
ind_neg = np.where(y_training == 'negative')
fig = plt.figure(1)
plt.bar(label,counts)
plt.xlabel("class label", labelpad=18)
plt.ylabel("number of documents", labelpad=18)
plt.title("Data Label Distribution", y=1.02);
fig.savefig(path+'/figures/label_distr.png')

train_text_length  = [len(str(text).split()) for text in training_data]

fig = plt.figure(2)
plt.hist(x=train_text_length,bins=80,
        color="steelblue",
        edgecolor="black")
plt.xlabel("text length", labelpad=18)
plt.ylabel("frequency", labelpad=18)
plt.title("Text Length Distribution", y=1.02);
fig.savefig(path+'/figures/total_text_len.png')

train_positive_text_length  = [len(str(text).split()) for text in training_data.iloc[list(ind_pos)[0]]]   
train_negative_text_length  = [len(str(text).split()) for text in training_data.iloc[list(ind_neg)[0]]]   

fig,ax = plt.subplots(1,2, figsize=(12,6))
fig.suptitle('Text Length')
ax[0].hist(x=train_positive_text_length,bins=80,color="steelblue",edgecolor="black")
ax[0].set(xlabel = 'Length', ylabel = 'frequency', title = 'Positive Reviews')
ax[1].hist(x=train_negative_text_length,bins=80,color="steelblue",edgecolor="black")
ax[1].set(xlabel = 'Length', ylabel = 'frequency', title = 'Negative Reviews')
fig.savefig(path+'/figures/text_len.png')

In [ ]:
positive_text_freq = training_data.iloc[list(ind_pos)[0]].str.split(expand=True).unstack().value_counts(ascending=True)
negative_text_freq = training_data.iloc[list(ind_neg)[0]].str.split(expand=True).unstack().value_counts(ascending=True)

fig,ax = plt.subplots(1,2, figsize=(12,13))
ax[0].barh(range(50),positive_text_freq[-50:], tick_label = positive_text_freq.keys()[-50:])
ax[0].set(title = '')
ax[1].barh(range(50),negative_text_freq[-50:], tick_label = negative_text_freq.keys()[-50:])
fig.savefig(path+'/figures/word frequency.png')

In [ ]:
num_words = 1500000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(training_data.tolist())
vocab = tokenizer.word_index

print(str(tokenizer.texts_to_sequences(['xyz how are you'])))

*import*

*Naive Bayes*



In [ ]:
pipe = Pipeline([('vector',CountVectorizer()),('NB',MultinomialNB())])
NB_model = pipe.fit(training_data,y_training)
NB_y_pred = pipe.predict(test_data)

In [ ]:
print(classification_report(y_test, NB_y_pred))

cm = confusion_matrix(y_test, NB_y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/NB cm.png")

In [ ]:
pipe = Pipeline([('vector',TfidfVectorizer()),('NB',MultinomialNB())])
NB_model = pipe.fit(training_data,y_training)
NB_y_pred = pipe.predict(test_data)

print(classification_report(y_test, NB_y_pred))

cm = confusion_matrix(y_test, NB_y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/NB_tfidf cm.png")

In [ ]:
# Cross Validation
parameters = {'vector__binary':[0,1], 'vector__ngram_range':[(1,1), (1,2),(2,2)],'NB__alpha':[0.1,1]}

Grid_S = GridSearchCV(pipe, param_grid = parameters,cv = 5)
res = Grid_S.fit(val_data,y_val)

In [ ]:
res.best_params_

In [ ]:
# optimal parameters

pipe = Pipeline([('vector',TfidfVectorizer(binary = 1, ngram_range=(1,2))),('NB',MultinomialNB(alpha = 1))])
NB_model = pipe.fit(training_data,y_training)
NB_y_pred = pipe.predict(test_data)

print(classification_report(y_test, NB_y_pred))

cm = confusion_matrix(y_test, NB_y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/NB_optimal cm.png")


In [ ]:
df_wrong_NB = test_data[NB_y_pred!=y_test]
print((df_wrong_NB.index))
print((df_wrong_NB))

*CNN*

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras import losses
from collections import Counter

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pydot
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
import sklearn.metrics

from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Conv1D, Flatten, Dense, Lambda, LSTM, concatenate,SimpleRNN,SpatialDropout1D
import keras.backend as K

from tensorflow.keras.optimizers import Adam

*plot dataset*

In [ ]:
#padding
X_train = np.array( tokenizer.texts_to_sequences(training_data) )
X_valid = np.array( tokenizer.texts_to_sequences(val_data) )
X_test  = np.array( tokenizer.texts_to_sequences(test_data.tolist()) )

X_train = pad_sequences(X_train, padding='post', maxlen=256)
X_valid = pad_sequences(X_valid, padding='post', maxlen=256)
X_test = pad_sequences(X_test, padding='post', maxlen=256)

le = LabelEncoder()

all_labels = le.fit_transform(y_training.to_list())
all_labels_onehot = np.asarray( tf.keras.utils.to_categorical(all_labels))
print(all_labels)
train_labels = le.transform(y_training)
train_labels_onehot = np.asarray( tf.keras.utils.to_categorical(train_labels))
print(train_labels.shape)

valid_labels = le.transform(y_val)
valid_labels_onehot = np.asarray( tf.keras.utils.to_categorical(valid_labels))

test_labels = le.transform(y_test.tolist())
test_labels_onehot = np.asarray(tf.keras.utils.to_categorical(test_labels))

train_ds = tf.data.Dataset.from_tensor_slices((X_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((X_test,test_labels))

In [ ]:
train_labels_bi = le.fit_transform(y_training)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels_bi))

pretrained embedding

In [ ]:
# read pretrained embedding
embeddings_index = {}
f = open(os.path.join( 'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# build embedding matrix
word_index = tokenizer.word_index
EMBEDDING_DIM = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [ ]:
def TextCNN_model_2(x_train_padded_seqs,y_train_onehot,x_valid_padded_seqs,y_valid_onehot,x_test_padded_seqs,y_test_onehot):

    main_input = Input(shape=(256,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = layers.Embedding(len(vocab) + 1, 100, input_length=256, weights=[embedding_matrix], trainable=True)
    embed = embedder(main_input)

    # 词窗大小分别为3,4,5
    cnn1 = layers.Conv1D(650, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = layers.MaxPooling1D(pool_size=198)(cnn1)
    cnn1 = layers.Dropout(0.5)(cnn1)
    cnn2 = layers.Conv1D(650, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = layers.MaxPooling1D(pool_size=196)(cnn2)
    cnn2 = layers.Dropout(0.5)(cnn2)
    cnn3 = layers.Conv1D(650, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = layers.MaxPooling1D(pool_size=194)(cnn3)
    cnn3 = layers.Dropout(0.5)(cnn3)
    cnn4 = layers.Conv1D(650, 6, padding='same', strides=1, activation='relu')(embed)
    cnn4 = layers.MaxPooling1D(pool_size=192)(cnn4)
    cnn4 = layers.Dropout(0.5)(cnn4)
    # 合并三个模型的输出向量
    cnn = layers.concatenate([cnn1, cnn2, cnn3,cnn4], axis=-1)
    flat = layers.Flatten()(cnn)
    drop = layers.Dropout(0.2)(flat)
    main_output = layers.Dense(2, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    optim = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()
    
    return model

model = TextCNN_model_2(X_train,train_labels_onehot,X_valid,valid_labels_onehot,X_test,test_labels_onehot)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint(path+'/best_model_cnn1.h5', monitor='val_acc', mode='max',
                     save_best_only=True, verbose=1)
    
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,valid_labels_onehot))

history = model.fit(X_train,
                    train_labels_onehot,
                    batch_size=400,
                    epochs=7,
                    validation_data = valid_ds.batch(400),
                    callbacks = [es,mc])

result = model.predict(X_test)  # 预测样本属于每个类别的概率
cnn1_result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签
print(classification_report(test_labels, cnn1_result_labels))
# plot_modelfit_process(history, "/training process CNN.png")
cm = confusion_matrix(test_labels, cnn1_result_labels)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/CNN1 cm.png")

In [ ]:
wrong_cnn = test_data[cnn1_result_labels!=test_labels]
print((wrong_cnn.index[:100]))
print((wrong_cnn))

In [ ]:
wrong_cnn = test_data[cnn1_result_labels!=test_labels]
print(wrong_cnn[18116])
print(IMDB_training_data['sentiment'][18116])
print(IMDB_training_data['review'][18116])

In [ ]:
#构建TextCNN模型
#模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接

def TextCNN_model_1(x_train_padded_seqs,y_train_onehot,x_valid_padded_seqs,y_valid_onehot,x_test_padded_seqs,y_test_onehot):
    main_input = Input(shape=(256,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = layers.Embedding(len(vocab) + 1, 100, input_length=256, trainable=True)
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn1 = layers.Conv1D(650, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = layers.MaxPooling1D(pool_size=196)(cnn1)
    cnn1 = layers.Dropout(0.5)(cnn1)
    cnn2 = layers.Conv1D(650, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = layers.MaxPooling1D(pool_size=194)(cnn2)
    cnn2 = layers.Dropout(0.5)(cnn2)
    cnn3 = layers.Conv1D(650, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = layers.MaxPooling1D(pool_size=192)(cnn3)
    cnn3 = layers.Dropout(0.5)(cnn3)
    cnn4 = layers.Conv1D(650, 6, padding='same', strides=1, activation='relu')(embed)
    cnn4 = layers.MaxPooling1D(pool_size=190)(cnn4)
    cnn4 = layers.Dropout(0.5)(cnn4)
    # 合并三个模型的输出向量
    cnn = layers.concatenate([cnn1, cnn2, cnn3,cnn4], axis=-1)
    flat = layers.Flatten()(cnn)
    drop = layers.Dropout(0.2)(flat)
    main_output = layers.Dense(2, activation='softmax')(drop)
    optim = Adam(lr=0.0005)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()

    return model

model = TextCNN_model_1(X_train,train_labels_onehot,X_valid,valid_labels_onehot,X_test,test_labels_onehot)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,valid_labels_onehot))

history = model.fit(X_train, 
                    train_labels_onehot, 
                    batch_size=400, 
                    epochs=3,
                    validation_data = valid_ds.batch(400),
                    callbacks = [es,mc])
result = model.predict(X_test)  # 预测样本属于每个类别的概率
cnn2_result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签

print(classification_report(test_labels, cnn2_result_labels))
# plot_modelfit_process(history, "/training process CNN2.png")
cm = confusion_matrix(test_labels, cnn2_result_labels)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/CNN2 cm.png")

In [ ]:
wrong_cnn = test_data[cnn2_result_labels!=test_labels]
print((wrong_cnn.index))
print((wrong_cnn))

In [ ]:
wrong_cnn = test_data[cnn2_result_labels!=test_labels]
print(wrong_cnn[12561])
# print(cnn1_result_labels[2])
print(IMDB_training_data['sentiment'][12561])
print(IMDB_training_data['review'][12561])

*RNN*

In [ ]:
def RNN_model(x_train_padded_seqs,y_train_onehot,x_valid_padded_seqs,y_valid_onehot,x_test_padded_seqs,y_test_onehot):
    main_input = Input(shape=(256,), dtype='float64')
    embedder = layers.Embedding(len(vocab) + 1, 100, input_length=256, trainable=True, weights=[embedding_matrix])
    embed = embedder(main_input)

    rnn1 = layers.Bidirectional(LSTM(128))(embed)
    drop = layers.Dropout(0.5)(rnn1)
    flat = layers.Flatten()(drop)
    dense1 = layers.Dense(768, activation="relu")(drop)
    drop2 = layers.Dropout(0.5)(dense1)
    dense2 = layers.Dense(2, activation="sigmoid")(drop2)

    optim = Adam(lr=0.0005)
    model = Model(inputs=main_input, outputs=dense2)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()

    return model
model = RNN_model(X_train,train_labels_onehot,X_valid,valid_labels_onehot,X_test,test_labels_onehot)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,valid_labels_onehot))

history = model.fit(X_train, train_labels_onehot, batch_size=400, epochs=6,validation_data = valid_ds.batch(400))
result = model.predict(X_test)  # 预测样本属于每个类别的概率
lstm_result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签

print(classification_report(test_labels, lstm_result_labels))
# plot_modelfit_process(history, "/training process RNN.png")
cm = confusion_matrix(test_labels, lstm_result_labels)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/RNN cm.png")

In [ ]:
wrong_lstm1 = test_data[lstm_result_labels!=test_labels]
print((wrong_lstm1.index))
print((wrong_lstm1))

In [ ]:
wrong_lstm1 = test_data[lstm_result_labels!=test_labels]
print(wrong_lstm1[12561])
print(IMDB_training_data['sentiment'][12561])
print(IMDB_training_data['review'][12561])

In [ ]:
def RNN2_model(x_train_padded_seqs,y_train_onehot,x_valid_padded_seqs,y_valid_onehot,x_test_padded_seqs,y_test_onehot):
    main_input = Input(shape=(256,), dtype='float64')
    embedder = layers.Embedding(len(vocab) + 1, 100, input_length=256, trainable=True)
    embed = embedder(main_input)

    rnn1 = layers.Bidirectional(LSTM(128))(embed)
    drop = layers.Dropout(0.5)(rnn1)
    flat = layers.Flatten()(drop)
    dense1 = layers.Dense(768, activation="relu")(drop)
    drop2 = layers.Dropout(0.5)(dense1)
    dense2 = layers.Dense(2, activation="sigmoid")(drop2)

    optim = Adam(lr=0.0005)
    model = Model(inputs=main_input, outputs=dense2)
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()

    return model
model = RNN2_model(X_train,train_labels_onehot,X_valid,valid_labels_onehot,X_test,test_labels_onehot)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid,valid_labels_onehot))

history = model.fit(X_train, train_labels_onehot, batch_size=400, epochs=3,validation_data = valid_ds.batch(400))
result = model.predict(X_test)  # 预测样本属于每个类别的概率
lstm2_result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签

print(classification_report(test_labels, lstm2_result_labels))
# plot_modelfit_process(history, "/training process RNN2.png")
cm = confusion_matrix(test_labels, lstm2_result_labels)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/RNN2 cm.png")

In [ ]:
wrong_lstm2 = test_data[lstm2_result_labels!=test_labels]
print((wrong_lstm2.index))
print((wrong_lstm2))

In [ ]:
wrong_lstm2 = test_data[lstm2_result_labels!=test_labels]
print(wrong_lstm2[12561])
# print(lstm2_result_labels[2])
print(IMDB_training_data['sentiment'][12561])
print(IMDB_training_data['review'][12561])

**Bert**

In [ ]:
def Encoder_BERT(text_data,labels):
  input_ids=[]
  attention_masks=[]
  sentences = text_data
  labels=labels
  len(sentences),len(labels)

  for sent in sentences:
      bert_inp=bert_tokenizer.encode_plus(sent,
                                          add_special_tokens = True,
                                          max_length =256,
                                          pad_to_max_length = True,
                                          return_attention_mask = True)
      input_ids.append(bert_inp['input_ids'])
      attention_masks.append(bert_inp['attention_mask'])

  input_ids=np.asarray(input_ids)
  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  return input_ids,attention_masks,labels

In [ ]:
tr_input_ids,tr_attention_masks,tr_labels = Encoder_BERT(training_data,y_training)
va_input_ids,va_attention_masks,va_labels = Encoder_BERT(val_data,y_val)
te_input_ids,te_attention_masks,te_labels = Encoder_BERT(test_data,y_test)

In [ ]:
a=le.fit_transform(tr_labels)

In [ ]:
import pickle
print('Preparing the pickle file.....')

# train validation test
tr_pickle_inp_path='tr_bert_inp.pkl'
tr_pickle_mask_path='tr_bert_mask.pkl'
tr_pickle_label_path='tr_bert_label.pkl'

va_pickle_inp_path='va_bert_inp.pkl'
va_pickle_mask_path='va_bert_mask.pkl'
va_pickle_label_path='va_bert_label.pkl'

te_pickle_inp_path='te_bert_inp.pkl'
te_pickle_mask_path='te_bert_mask.pkl'
te_pickle_label_path='te_bert_label.pkl'

pickle.dump((tr_input_ids),open(tr_pickle_inp_path,'wb'))
pickle.dump((tr_attention_masks),open(tr_pickle_mask_path,'wb'))
pickle.dump((tr_labels),open(tr_pickle_label_path,'wb'))

pickle.dump((va_input_ids),open(va_pickle_inp_path,'wb'))
pickle.dump((va_attention_masks),open(va_pickle_mask_path,'wb'))
pickle.dump((va_labels),open(va_pickle_label_path,'wb'))

pickle.dump((te_input_ids),open(te_pickle_inp_path,'wb'))
pickle.dump((te_attention_masks),open(te_pickle_mask_path,'wb'))
pickle.dump((te_labels),open(te_pickle_label_path,'wb'))

# print('train Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

In [ ]:
# Load Pickles
import pickle

tr_pickle_inp_path='tr_bert_inp.pkl'
tr_pickle_mask_path='tr_bert_mask.pkl'
tr_pickle_label_path='tr_bert_label.pkl'

va_pickle_inp_path='va_bert_inp.pkl'
va_pickle_mask_path='va_bert_mask.pkl'
va_pickle_label_path='va_bert_label.pkl'

te_pickle_inp_path='te_bert_inp.pkl'
te_pickle_mask_path='te_bert_mask.pkl'
te_pickle_label_path='te_bert_label.pkl'

print('Loading the saved pickle files..')

tr_input_ids=pickle.load(open(tr_pickle_inp_path, 'rb'))
tr_attention_masks=pickle.load(open(tr_pickle_mask_path, 'rb'))
tr_labels=pickle.load(open(tr_pickle_label_path, 'rb'))

va_input_ids=pickle.load(open(va_pickle_inp_path, 'rb'))
va_attention_masks=pickle.load(open(va_pickle_mask_path, 'rb'))
va_labels=pickle.load(open(va_pickle_label_path, 'rb'))

te_input_ids=pickle.load(open(te_pickle_inp_path, 'rb'))
te_attention_masks=pickle.load(open(te_pickle_mask_path, 'rb'))
te_labels=pickle.load(open(te_pickle_label_path, 'rb'))

print('Training Input shape {} Training Attention mask shape {} Training Input label shape {}'.format(tr_input_ids.shape,tr_attention_masks.shape,tr_labels.shape))
print('Validation Input shape {} Validation  Attention mask shape {} Validation  Input label shape {}'.format(va_input_ids.shape,va_attention_masks.shape,va_labels.shape))
print('Test Input shape {} Test Attention mask shape {} Test Input label shape {}'.format(te_input_ids.shape,te_attention_masks.shape,te_labels.shape))
print(te_labels)

In [ ]:
import pickle

pickle_inp_path='bert_inp.pkl'
pickle_mask_path='bert_mask.pkl'
pickle_label_path='bert_label.pkl'

print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))
print(labels)

In [ ]:
log_dir='tb_bert'
model_save_path='bert_model.h5'
import keras
# import tensorflow.keras.utils.plot_model

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nBert Model',bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

tf.keras.utils.plot_model(bert_model, 
                          # to_file='/content/drive/MyDrive/Colab Notebooks/TextMining/bertmodel_plot.png', 
           show_shapes=True, 
           show_layer_names=False)

In [ ]:
history=bert_model.fit([tr_input_ids,tr_attention_masks],le.fit_transform(tr_labels),batch_size=16,epochs=10,validation_data=([va_input_ids,va_attention_masks],le.fit_transform(va_labels)),callbacks=callbacks)
plot_modelfit_process(history, "/training process BERT.png")

In [ ]:
model_save_path='bert_model.h5'

trained_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

preds = trained_model.predict([te_input_ids,te_attention_masks],batch_size=32)
pred_labels = np.argmax(preds[0],axis=1)
# f1 = f1_score(val_label, pred_labels)
# print('F1 score',f1)
print('Classification Report')
print(classification_report(le.fit_transform(te_labels),pred_labels))
print('Training and saving built model.....')  

cm = confusion_matrix(test_labels, pred_labels)
display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "positive"])
display.plot()
plt.show()
display.im_.figure.savefig(path+"/BERT cm.png")

In [ ]:
wrong_bert = test_data[pred_labels!=test_labels]
print((wrong_bert.index))
print((wrong_bert))

In [ ]:
wrong_bert = test_data[pred_labels!=test_labels]
print(wrong_bert[12561])
# print(lstm2_result_labels[2])
print(IMDB_training_data['sentiment'][12561])
print(IMDB_training_data['review'][12561])